In [1]:
import json
import ast
import re
import csv

from pandas import read_csv

# 1. Chuyển từ dữ liệu text thành file csv

In [2]:
def extract_json_blocks(text: str):
    blocks = []

    pattern = r"```(?:json|python)?\s*(.*?)```"
    matches = re.findall(pattern, text, flags=re.DOTALL)

    for m in matches:
        candidate = m.strip()
        if ("[" in candidate and "]" in candidate) or ("{" in candidate and "}" in candidate):
            blocks.append(candidate)

    if not blocks:
        cleaned = text.strip()
        if cleaned:
            blocks = [cleaned]

    return blocks



In [3]:
def parse_block_to_python(block: str):
    block = block.strip()

    try:
        return json.loads(block)
    except Exception:
        pass

    try:
        return ast.literal_eval(block)
    except Exception:
        pass

    try:
        wrapped = "[" + block + "]"
        return json.loads(wrapped)
    except Exception:
        pass

    print(block)

    raise ValueError("Không parse được block:\n" + block[:200])

In [4]:
def normalize_record(rec: dict):
    def to_list(v):
        if v is None:
            return []
        if isinstance(v, list):
            return v
        if isinstance(v, dict):
            return [v]
        return [v]

    mapping = {
        "Tên cây": "ten_cay",
        "tên cây": "ten_cay",
        "Loại bệnh": "loai_benh",
        "loại bệnh": "loai_benh",
        "Nguyên nhân": "nguyen_nhan",
        "nguyên nhân": "nguyen_nhan",
        "Triệu chứng": "trieu_chung",
        "triệu chứng": "trieu_chung",
        "Cách điều trị": "cach_dieu_tri",
        "cách điều trị": "cach_dieu_tri",
    }

    out = {
        "ten_cay": "",
        "loai_benh": "",
        "nguyen_nhan": [],
        "trieu_chung": [],
        "cach_dieu_tri": []
    }

    for k, v in rec.items():
        if k in mapping:
            mk = mapping[k]
            if mk in ["ten_cay", "loai_benh"]:
                out[mk] = v.strip() if isinstance(v, str) else str(v)
            else:
                out[mk] = to_list(v)
        else:
            pass

    return out

In [5]:
def pipeline(txt_path: str, json_path: str, csv_path: str):
    with open(txt_path, "r", encoding="utf-8") as f:
        raw = f.read()

    blocks = extract_json_blocks(raw)

    results = []

    for block in blocks:
        if block.startswith("Dưới đây") or block.startswith("Dựa trên nội dung") or block.startswith("Từ nội dung") or block.startswith("Dựa vào nội dung"):
            print(block)
            continue
        print("---------------")
        parsed = parse_block_to_python(block)
        if isinstance(parsed, list):
            for item in parsed:
                if isinstance(item, dict):
                    results.append(normalize_record(item))
        elif isinstance(parsed, dict):
            results.append(normalize_record(parsed))
    with open(csv_path, "a", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["ten_cay", "loai_benh", "nguyen_nhan", "trieu_chung", "cach_dieu_tri"]
        )
        writer.writeheader()
        for r in results:
            writer.writerow({
                "ten_cay": r["ten_cay"],
                "loai_benh": r["loai_benh"],
                "nguyen_nhan": " | ".join(map(str, r["nguyen_nhan"])),
                "trieu_chung": " | ".join(map(str, r["trieu_chung"])),
                "cach_dieu_tri": " | ".join(map(str, r["cach_dieu_tri"])),
            })

    print(f"✔ Hoàn thành! Tổng số record: {len(results)}")
    print(f"→ JSON: {json_path}")
    print(f"→ CSV:  {csv_path}")

In [11]:
pipeline(
        txt_path="raw_data/raw_data_general_2.txt",
        json_path="clean_data_test1.json",
        csv_path="data/general_data_raw_3.csv"
    )


---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
--------

# 2. Tiền xử lý dữ liệu sau khi đã được chuyển đổi

## Tìm kiếm những bản ghi có cột loại bệnh và tên cây trùng nhau và loại bỏ

In [21]:
# Tìm kiếm các row có tên cây và loại bệnh trùng nhau
def find_duplicates(csv_path):
    df = pd.read_csv(csv_path)

    # Tìm duplicated theo 2 cột
    dup_mask = df.duplicated(subset=["ten_cay", "loai_benh"], keep=False)

    duplicates_df = df[dup_mask].sort_values(by=["ten_cay", "loai_benh"])

    return duplicates_df

In [22]:
dups = find_duplicates("data/data_clean_1.csv")
dups  # xem các hàng trùng nhau

,ten_cay,loai_benh,nguyen_nhan,trieu_chung,cach_dieu_tri
67,cao su,Bệnh nấm hồng,Nấm Corticium salmonicolor. Phát triển mạnh tr...,Giai đoạn đầu trên bề mặt vỏ cây xuất hiện lớp...,"Phòng ngừa: Chọn giống kháng bệnh, canh tác đú..."
99,cao su,Bệnh nấm hồng,Tác nhân chính: Nấm Corticium salmonicolor. | ...,{'Trên thân và cành cây': ['Ban đầu: Các sợi n...,{'Biện pháp canh tác phòng trừ': ['Xây dựng hệ...
63,cao su,Bệnh phấn trắng,Do nấm Oidium heveae Steinm hoặc các loại nấm ...,Giai đoạn đầu (lá non màu đồng tím): Lá cây nh...,"{'phòng ngừa và canh tác': ['Tỉa cành, vệ sinh..."
66,cao su,Bệnh phấn trắng,Nấm Oidium Hevea. Phát triển mạnh trong thời t...,"Lá non có màu đồng tím, nhăn nheo và có lớp bộ...","Phòng ngừa: Chọn giống kháng bệnh, canh tác đú..."
637,chanh,Đốm lá,Nấm gây hại (thuốc Phy Fusaco có công dụng điề...,Không được mô tả chi tiết trong bài viết.,"{'Tên thuốc': 'Phy Fusaco', 'Hướng dẫn sử dụng..."
...,...,...,...,...,...
539,đậu cove,Đốm lá,"Không được đề cập chi tiết trong bài viết, như...",Không được đề cập chi tiết trong bài viết.,Phòng ngừa và điều trị bằng thuốc sinh học Ant...
548,đậu cove,Đốm lá,Do nấm Cercospora cruenta xâm nhập và gây hại....,"Trên lá: Xuất hiện các vết đốm vòng, trung tâm...",{'phương pháp chung': ['Thu gom tàn dư thực vậ...
135,ớt,Bệnh thán thư,Do nấm Collectotrichum (đặc biệt là Collectotr...,{'Tổng quát': 'Có thể xuất hiện trên bất kỳ bộ...,{'Phòng ngừa': ['Chọn thời vụ trồng hợp lý (ít...
948,ớt,Bệnh thán thư,Không đề cập chi tiết trong bài viết (chỉ liệt...,Không đề cập chi tiết trong bài viết.,Không đề cập chi tiết trong bài viết.


In [2]:
import pandas as pd

def clean_ten_cay(name: str) -> str:
    """
    Xóa từ 'cây' hoặc 'cây ' ở đầu chuỗi.
    Không ảnh hưởng các vị trí khác.
    Ví dụ:
        'cây hoa cúc' -> 'hoa cúc'
        'cây cam' -> 'cam'
        'hoa hồng' -> 'hoa hồng' (giữ nguyên)
    """
    if not isinstance(name, str):
        return name

    name = name.strip().lower()

    # Nếu bắt đầu bằng 'cây '
    if name.startswith("cây "):
        return name[4:].strip()  # bỏ 'cây ' + strip dư thừa

    # Nếu chỉ là đúng chữ 'cây'
    if name == "cây":
        return ""

    return name

# clean loại bệnh có chữ bệnh ở trước
def clean_loai_benh(name: str) -> str:
    if not isinstance(name, str):
        return name

    name = name.strip().lower()

    # Nếu bắt đầu bằng 'cây '
    if name.startswith("bệnh "):
        return name[4:].strip()  # bỏ 'cây ' + strip dư thừa

    # Nếu chỉ là đúng chữ 'cây'
    if name == "bệnh":
        return ""

    return name

def remove_duplicates_keep_first(df):
    df_cleaned = df.drop_duplicates(
        subset=["ten_cay", "loai_benh"],
        keep="first"   # giữ dòng đầu tiên
    )

    return df_cleaned

df = pd.read_csv("data/general_data_raw_3.csv")

df["ten_cay"] = df["ten_cay"].apply(clean_ten_cay)
df["loai_benh"] = df["loai_benh"].apply(clean_loai_benh)

df_cleaned = remove_duplicates_keep_first(df)

df_cleaned.to_csv("data/data_clean_3.csv", index=False)

## Chuẩn hóa 3 cột nguyên nhân, triệu chứng và cách điều trị

In [3]:
import pandas as pd
import ast

def try_parse_json(value):
    """Chuyển chuỗi dạng JSON thành dict nếu có thể."""
    if not isinstance(value, str):
        return value
    try:
        return ast.literal_eval(value)
    except:
        return value

In [10]:
def is_valid_value(text: str) -> bool:
    """
    Kiểm tra xem một giá trị đã tách ra có hợp lệ hay không.
    - Loại bỏ text rỗng
    - Loại bỏ text 1-3 ký tự (S, d, ụ,...)
    - Loại bỏ text không có chữ cái
    """
    if not isinstance(text, str):
        return False

    t = text.strip()

    if t == "":
        return False

    # Loại chuỗi quá ngắn
    if len(t) < 4:
        return False

    # Nếu không chứa ký tự chữ → loại
    if not re.search(r"[A-Za-zÀ-ỹ]", t):
        return False

    # Loại chuỗi toàn ký tự đặc biệt
    if all(ch in "-|•*.,/" for ch in t):
        return False

    return True

In [20]:
def clean_text(text: str) -> str:
    """
    Xóa các ký tự đặc biệt thường gặp trong dữ liệu gốc
    như *, •, |, -, ⋅, v.v...
    """
    if not isinstance(text, str):
        return text

    # Xóa ký tự đặc biệt
    text = re.sub(r"[\*\•\|\⋅]", " ", text)

    # Xóa khoảng trắng dư
    return ' '.join(text.split())

In [21]:
def flatten_json_dict(entity, relation_prefix, data_dict):
    """
    Chuyển JSON dict thành list các atomic records.
    """
    rows = []
    for key, values in data_dict.items():
        clean_key = (
            key.lower()
               .replace(" ", "_")
               .replace("–", "_")
               .replace("-", "_")
               .replace("và", "_")
                )

        for v in values:
            if is_valid_value(v):
                rows.append({
                    "entity": entity,
                    "relation": f"{relation_prefix}_{clean_key}",
                    "value": v.strip()
                })
    return rows

In [22]:
def split_text_list(entity, relation, text):
    items = re.split(r"[\-\|•\n]+", text)
    rows = []
    for item in items:
        item = clean_text(item.strip())
        if len(item) > 3:
            rows.append({
                "entity": entity,
                "relation": relation,
                "value": item
            })
    return rows

In [23]:
def process_for_kg(csv_path):
    df = pd.read_csv(csv_path)

    result_rows = []

    for _, row in df.iterrows():
        entity = row["ten_cay"]

        for col in ["nguyen_nhan", "trieu_chung", "cach_dieu_tri"]:
            raw = row[col]
            parsed = try_parse_json(raw)

            if isinstance(parsed, dict):
                # JSON dict → flatten
                result_rows.extend(flatten_json_dict(entity, col, parsed))

            else:
                # text dạng bullet → split
                clean_raw = clean_text(str(raw))
                result_rows.extend(split_text_list(entity, col, clean_raw))

    return pd.DataFrame(result_rows)

In [24]:
df_test = process_for_kg("data/test_data.csv")

In [25]:
df_test

,entity,relation,value
0,Cây mai vàng,nguyen_nhan,"Do các loại nấm bệnh tấn công, đặc biệt là nấm..."
1,Cây mai vàng,trieu_chung,"Nấm bệnh thường phát sinh vào cuối mùa thu, tr..."
2,Cây mai vàng,cach_dieu_tri_phương_pháp_canh_tác_phòng_trừ,"Lựa chọn giống mai tốt, ít bị nấm và sâu bệnh,..."
3,Cây mai vàng,cach_dieu_tri_phương_pháp_canh_tác_phòng_trừ,"Thường xuyên theo dõi và chăm sóc vườn cây, xâ..."
4,Cây mai vàng,cach_dieu_tri_phương_pháp_canh_tác_phòng_trừ,Duy trì chế độ tưới đều đặn (sáng sớm hoặc chi...
...,...,...,...
130,Cây cà phê,trieu_chung,"Vết bệnh xuất hiện ở mặt dưới của lá, bắt đầu ..."
131,Cây cà phê,cach_dieu_tri,Xem mục 'cách điều trị' của 'Nấm hồng' vì bài ...
132,Cây cà phê,nguyen_nhan,Nấm bệnh thường bắt đầu phát triển vào mùa mưa...
133,Cây cà phê,trieu_chung,Trái cà phê bắt đầu xuất hiện triệu chứng thối...


In [19]:
df_test.to_csv("data/out_test.csv", index=False)

In [38]:
import ast
import json
import re
import unicodedata
import pandas as pd
from typing import Any

# -------------------------
# Helper: clean text
# -------------------------
def _clean_text_simple(s: str) -> str:
    """Chuẩn hoá unicode, loại ký tự đặc biệt, trim nhiều khoảng trắng."""
    if not isinstance(s, str):
        return s
    # Unicode NFC
    s = unicodedata.normalize("NFC", s)
    # Thay các ký tự đặc biệt thường gặp bằng khoảng trắng
    s = re.sub(r"[\*\•\|\⋅\•\u2022\u2023\u25E6\u2043]", " ", s)
    # Loại các ký tự kiểm soát lạ
    s = re.sub(r"[\x00-\x1F\x7F]", " ", s)
    # Thay nhiều dấu cách bằng 1
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _is_meaningful_token(tok: str, min_len: int = 4) -> bool:
    """Trả True nếu token đủ dài và có ký tự chữ."""
    if not isinstance(tok, str):
        return False
    t = tok.strip()
    if len(t) < min_len:
        return False
    # Có ít nhất 1 ký tự alphabet (viết thường/hoa hoặc chữ Việt)
    return bool(re.search(r"[A-Za-zÀ-ỹ]", t))

def _safe_parse(value: Any):
    if value is None:
        return None
    if isinstance(value, (dict, list)):
        return value
    if not isinstance(value, str):
        return value
    v = value.strip()
    if v == "":
        return ""

    try:
        return json.loads(v)
    except Exception:
        pass
    # Try python literal (dict/list written like Python)
    try:
        return ast.literal_eval(v)
    except Exception:
        pass
    # fallback: original string
    return v

# -------------------------
# Flatten any value -> clean string
# -------------------------
def _flatten_value(value: Any) -> str:
    """
    Chuyển value (str/dict/list) thành 1 chuỗi sạch:
    - dict -> "Key1: val1; Key2: val2"
    - list -> "item1; item2"
    - str -> clean và giữ nguyên (loại bỏ token ngắn)
    """
    if value is None:
        return ""

    # Nếu là dict: duyệt key->value
    if isinstance(value, dict):
        parts = []
        for k, v in value.items():
            # convert v -> list of strings
            if isinstance(v, list):
                items = []
                for it in v:
                    itc = _clean_text_simple(str(it))
                    if _is_meaningful_token(itc):
                        items.append(itc)
                if items:
                    parts.append(f"{_clean_text_simple(str(k))}: " + "; ".join(items))
            else:
                # v có thể là str
                vc = _clean_text_simple(str(v))
                if _is_meaningful_token(vc):
                    parts.append(f"{_clean_text_simple(str(k))}: {vc}")
        return " | ".join(parts).strip()

    # Nếu là list: nối các phần ý nghĩa
    if isinstance(value, list):
        items = []
        for it in value:
            itc = _clean_text_simple(str(it))
            if _is_meaningful_token(itc):
                items.append(itc)
        return "; ".join(items)

    # Nếu là string: có thể chứa các mục con (dạng bullet). Tách và lọc token ngắn.
    if isinstance(value, str):
        # tách theo dấu bullet/dấu phẩy/dấu chấm phẩy/dòng mới
        tokens = re.split(r"[\n\r;•\|\-\u2022]+", value)
        tokens = [ _clean_text_simple(t) for t in tokens if t and _clean_text_simple(t) ]
        # giữ token có nghĩa
        tokens = [t for t in tokens if _is_meaningful_token(t)]
        if tokens:
            return "; ".join(tokens)
        # nếu không có token dài (ví dụ toàn chữ ngắn), giữ nguyên string sạch nếu đủ dài
        s_clean = _clean_text_simple(value)
        return s_clean if _is_meaningful_token(s_clean) else ""

    # else fallback
    return _clean_text_simple(str(value))

# -------------------------
# Main: process DataFrame
# -------------------------
def process_three_columns(df: pd.DataFrame,
                          cols_to_fix = ("nguyen_nhan", "trieu_chung", "cach_dieu_tri")) -> pd.DataFrame:
    """
    Trả về DataFrame copy với 3 cột đã được parse & flatten thành chuỗi sạch.
    Không thay đổi các cột khác; giữ nguyên 5 cột cấu trúc ban đầu.
    """
    df_out = df.copy(deep=True)

    for col in cols_to_fix:
        if col not in df_out.columns:
            continue
        fixed_values = []
        for raw in df_out[col].fillna("").tolist():
            parsed = _safe_parse(raw)
            flattened = _flatten_value(parsed)
            fixed_values.append(flattened)
        df_out[col] = fixed_values

    return df_out

# -------------------------
# Example usage:
# -------------------------
df_input = pd.read_csv("data/data_clean_3.csv")  # hoặc df hiện có
df_cleaned = process_three_columns(df_input)
df_cleaned.to_csv("data/data_general_final.csv", index=False)

In [36]:
df_cleaned

,ten_cay,loai_benh,nguyen_nhan,trieu_chung,cach_dieu_tri
0,Cây mai vàng,Bệnh cháy lá (hay còn gọi là cháy bìa lá),"Do các loại nấm bệnh tấn công, đặc biệt là nấm...","Nấm bệnh thường phát sinh vào cuối mùa thu, tr...",Phương pháp canh tác phòng trừ: Lựa chọn giống...
1,Sầu riêng,Bệnh phấn trắng,Do nấm Oidium sp. gây ra.; Nấm phát triển thuậ...,Qua lá: Xuất hiện lớp bụi màu trắng mịn và dày...,Biện pháp phòng ngừa:; Chọn giống sầu riêng kh...
2,Sầu riêng,Khô cành (còn gọi là khô đọt),Tác nhân chính: Nấm khuẩn Rhizoctonia solani.;...,"Bệnh xuất hiện từ cành nhỏ, cành phía dưới tán...",{'Loại biện pháp': 'Kỹ thuật canh tác phòng bệ...
3,Khoai mì (sắn),Thối củ (còn gọi là lở cổ rễ),Trực tiếp: Do nấm Phytopythium helicoides gây ...,Xuất hiện trên vị trí cổ rễ (phần thân sát gốc...,Khi phát hiện bệnh (xử lý ngay): Nhổ bỏ toàn b...
4,Cây dừa,Cháy lá,Do nấm bệnh Pestalozzia palmarum và Helminthos...,Khi nhiễm nấm Pestalozzia palmarum: Trên lá xu...,Canh tác phòng ngừa:; Bổ sung dinh dưỡng: Bón ...
5,Cây dừa,Nứt thân xì mủ,Không được nêu cụ thể trong bài viết cho cây d...,Không được nêu cụ thể trong bài viết cho cây dừa.,Sử dụng thuốc sinh học Phy FusaCo (tham khảo h...
6,Cây dừa,Thán thư,Không được nêu cụ thể trong bài viết cho cây d...,Không được nêu cụ thể trong bài viết cho cây dừa.,Sử dụng thuốc sinh học Phy FusaCo (tham khảo h...
7,Cây dừa,Ghẻ loét,Không được nêu cụ thể trong bài viết cho cây d...,Không được nêu cụ thể trong bài viết cho cây dừa.,Sử dụng thuốc sinh học Phy FusaCo (tham khảo h...
8,Cây dừa,Thối thân,Không được nêu cụ thể trong bài viết cho cây d...,Không được nêu cụ thể trong bài viết cho cây d...,Sử dụng thuốc sinh học Phy FusaCo (tham khảo h...
9,Cây dừa,Thối gốc,Không được nêu cụ thể trong bài viết cho cây d...,Không được nêu cụ thể trong bài viết cho cây dừa.,Sử dụng thuốc sinh học Phy FusaCo (tham khảo h...
